In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime

In [1]:
from function_set import *

In [3]:
anue_url = 'https://api.cnyes.com/media/api/v1/newslist/category/headline'
eco_url = 'https://money.udn.com/rank/newest/1001/0/1?from=edn_navibar'
ctee_url = 'https://ctee.com.tw/livenews/aj'


In [3]:
def get_anue_titles_urls(url, pages:str, start:str = str(int(datetime.datetime(2023, 2, 19).timestamp())), end:str =str(int(datetime.datetime(2023, 2, 21).timestamp())-1)) :
    pages = get_anue_total_page(url, pages, start, end)
    data = pd.DataFrame()
    for i in range(1, pages+1):
        response = request_url(url, i, start, end)
        temp = get_titles(response.json())
        data = pd.concat([data, temp])
    data = data.reset_index(drop=True)
    data['news_urls'] = data['news_urls'].apply(get_anue_news_urls)
    return data

def request_url(url, pages:str, start:str = str(int(datetime.datetime(2023, 2, 19).timestamp())), end:str =str(int(datetime.datetime(2023, 2, 21).timestamp())-1)):
    payload = {
    'limit':'30',
    'startAt': start,
    'endAt': end,
    'page': pages,
    'startAt': start,
    'endAt': end,
    'limit': '30'
    }
    response = rq.get(url, params=payload)
    return response

def get_titles(data):
    news_urls = []
    news_titles = []
    for i in data['items']['data']:
        news_urls += [i['newsId']]
        news_titles += [i['title']]
    data = {
        'news_urls':news_urls,
         'news_titles':news_titles
         }
    return pd.DataFrame(data)

def get_anue_total_page(url, pages, start:str = str(int(datetime.datetime(2023, 2, 19).timestamp())), end:str =str(int(datetime.datetime(2023, 2, 21).timestamp())-1)):
    response = request_url(url, pages, start, end)
    data = response.json()
    total = data['items']['total']
    per_page = data['items']['per_page']
    return int(total/per_page)

def get_anue_news_urls(word):
    prefix = 'https://news.cnyes.com/news/id/'
    return prefix + str(word)
    



In [6]:
data_anue = get_anue_titles_urls(anue_url, '1')
data_anue

In [32]:
def get_html(url):
    response = rq.get(url)
    soup = bs(response.text, 'html.parser')
    return soup

def crawl_ctee_titles(url):
    data = pd.DataFrame()
    while len(data) == 0:
        soup = get_html(url)
        links = soup.find_all('a',{'style':'color:#777777;text-decoration: none;'})
        urls = []
        titles = []
        for link in links:
            urls.append(link.get('href'))
            titles.append(link.text.strip())
        data = {
            'news_urls':urls,
            'news_titles':titles
        }
        data = pd.DataFrame(data)

    return data

def get_ctee_pages(url):
    final_pages = 0
    while final_pages == 0:
        soup = get_html(url)
        for i in soup.find_all('a',{'class':'page-numbers'}):
            if i.text != '下一頁 »':
                final_pages = i.text
        final_pages = int(final_pages)
    return final_pages

def get_ctee_titles_urls(url):
    total_pages = get_ctee_pages(url)
    data = pd.DataFrame()
    for i in range(1, total_pages+1):
        if i == 1:
            temp = crawl_ctee_titles(url)
        else:
            new_url = url +'/page/'+str(i)
            temp = crawl_ctee_titles(new_url)
        data = pd.concat([data, temp])
    return data


In [37]:
data_ctee = get_ctee_titles_urls(ctee_url)
data_ctee

,news_urls,news_titles
0,https://ctee.com.tw/livenews/aj/ctee/A93610002...,保誠集團新任執行總裁Anil Wadhwani上任 | 02/23 17:44
1,https://ctee.com.tw/livenews/aj/ctee/A99634002...,高雄科工館房市夯 歐美建設推森青道攻換屋客 | 02/23 17:39
2,https://ctee.com.tw/livenews/aj/ctee/A93610002...,跨品牌、易操作、彈性高 鉅亨母子基金「鉅寶盆」登場 | 02/23 17:38
3,https://ctee.com.tw/livenews/aj/ctee/A03619002...,市場拉貨動能回溫 南紡：第二季可望谷底翻 | 02/23 17:38
4,https://ctee.com.tw/livenews/aj/ctee/A81256002...,文化部砸22.5億振興藝文產業 擴大文化成年禮金 | 02/23 17:32
...,...,...
45,https://ctee.com.tw/livenews/aj/chinatimes/202...,《國際金融》匯豐扯後腿 歐股早市疲弱 | 02/21 17:09
46,https://ctee.com.tw/livenews/aj/chinatimes/202...,《金融》富邦＆日盛雙證與雙期 訂4月9日為合併基準日 | 02/21 17:09
47,https://ctee.com.tw/livenews/aj/chinatimes/202...,《電子通路》迎港澳客 燦坤推優惠 | 02/21 17:09
48,https://ctee.com.tw/livenews/aj/ctee/A93610002...,玉山金控入選2023彭博性別平等指數 深獲國際肯定 | 02/21 17:08


In [38]:
data = pd.concat([data_anue, data_ctee]).reset_index(drop=True)
data

,news_urls,news_titles
0,https://news.cnyes.com/news/id/5091817,中國人大再發聲明 批「氣球決議案」是抺黑 炒作「中國威脅」
1,https://news.cnyes.com/news/id/5091783,華為江淮新能源廠開工 總投資約26億元人民幣 估2024年初完工
2,https://news.cnyes.com/news/id/5091799,美中關係重新趨緊？中國官媒發布長文痛批美國「霸權、霸道、霸凌」
3,https://news.cnyes.com/news/id/5091782,馬斯克：OpenAI 和我創建它的初衷已背道而馳
4,https://news.cnyes.com/news/id/5091796,獨董杜英達、陳敏薰控管理層違反公司治理 泰山駁勿信口雌黃
...,...,...
535,https://ctee.com.tw/livenews/aj/chinatimes/202...,《國際金融》匯豐扯後腿 歐股早市疲弱 | 02/21 17:09
536,https://ctee.com.tw/livenews/aj/chinatimes/202...,《金融》富邦＆日盛雙證與雙期 訂4月9日為合併基準日 | 02/21 17:09
537,https://ctee.com.tw/livenews/aj/chinatimes/202...,《電子通路》迎港澳客 燦坤推優惠 | 02/21 17:09
538,https://ctee.com.tw/livenews/aj/ctee/A93610002...,玉山金控入選2023彭博性別平等指數 深獲國際肯定 | 02/21 17:08


In [55]:
data[data['news_titles'].duplicated()]

,news_urls,news_titles


## 匯入 excel

In [45]:
import openpyxl

In [50]:
# 讀取 excel
workbook = openpyxl.load_workbook('試題.xlsx')
# 選取 試題一 的工作表
sheet = workbook.worksheets[1]
col = ['K']
for index in range(300-13):
    link = data.loc[index, 'news_urls']
    text = data.loc[index, 'news_titles']
    sheet.cell(row=index+13,column=7,value=text)
    sheet.cell(row=index+13,column=7).hyperlink=link

In [51]:
# 儲存檔案
workbook.save('test1.xlsx')